[Theano](http://deeplearning.net/software/theano/index.html) is an amazing piece of software.
It allows to describe a mathematical expression by composing it from simpler, pre-defined components.
This high-level description has two main benefits: 
- the framework can automatically derive gradients using the structure of the composition, making theano particularly suitable to specify complex optimization objectives
- theano expressions can be executed on different backends, in particular GPUs that can lead to speedups of 1-2 orders of magnitude

These features make Theano particularly suitable for deep learning, since deep neural networks consists of several layers of similar structure. The training procedure can often be posed as a continuous, unconstrained optimization problem, and training large networks on huge amounts of data require 

In [93]:
import theano
import theano.tensor as T
from numpy import *

def stepT(txt, thprev, tWh, tWi, sigma=T.nnet.relu):
    tat = T.dot(tWh, thprev) + tWi[txt]
    tht = sigma(tat)
    return tht

def relu(x):
    return maximum(0.0, x)

def step(xt, hprev, Wh, Wi, sigma=relu):
    at = dot(Wh, hprev) + Wi[xt]
    ht = sigma(at)
    return ht

We are using the following version of theano

```python
print theano.version.version
```
    0.8.2.dev-RELEASE
 

In [94]:
random.seed(20160825)

D = 128
M = pow(2, 16)

D = 2
M = 3
B = 10

Wo = random.randn(D,M).astype(float32)
Wh = random.randn(D,D).astype(float32)
Wi = random.randn(M,D).astype(float32)
h0 = random.rand(D).astype(float32) 
seq = random.randint(0, M, B+1).astype(int32)
x = seq[:-1]
y = seq[1:]

In [98]:
h = h0
H_np = zeros((B, D)).astype(float32)
for (t,xt) in enumerate(x):
    h = step(xt, h, Wh, Wi)
    H_np[t] = h
print H_np - H_Tscan

[[ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]]


In [95]:
th0 = T.vector()
tWh = T.matrix()
tWi = T.matrix()
tx = T.ivector()

In [96]:
tH, tUp = theano.scan(fn = stepT, outputs_info=[th0], sequences=[tx], non_sequences=[tWh,tWi])

In [97]:
H_Tscan = tH.eval({tx:x, tWh:Wh, tWi:Wi, th0:h0})

In [101]:
th = th0
tH_tmp = []
for (t,xt) in enumerate(x):
    th = stepT(tx[t], th, tWh, tWi)
    tH_tmp.append(th)
tH_Tunroll = T.stack(tH_tmp)

H_Tunroll = tH_Tunroll.eval({tx:x, tWh:Wh, tWi:Wi, th0:h0})

In [102]:
print H_np - H_Tunroll

[[ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]]
